
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


<strong>Imports :

In [2]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


import cv2
import time

<strong> Variance for position :

In [3]:
qpx = 0.25 # variance on position state
qpy = 0.25
rpx = 0.04 # variance on position measurement 
rpy = 0.04

qpx = 0.25 # variance on position state
qpy = 0.25 
rpx = 0.01 # variance on position measurement 
rpy = 0.01

std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = 0.1# std_speed/2 # variance on speed measurement 

# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor_straight = 0.338; #straight
speed_conv_factor_left = 0.403;
speed_conv_factor_right = 0.425;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels


In [4]:
from globalNavigation import *
from helpers import *
from vision import *

## Main (from her on, keep in notebook)

In [5]:
# set up thymio

from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 0102d6c0-0c6c-479f-8406-474092b0daba

In [6]:
# Open the video capture
cap = cv2.VideoCapture(0)

2023-12-07 19:06:00.235 Python[11417:571261] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [7]:
async def path_following(motor_speed=100, threshold=0,  verbose=False):
    
    global n, positions, x_est, P_est, Ts, read, Vr, Vl, n

    saw_obstacle = False
    
    if verbose: print("\t Path following ")
    await node.set_variables(motors(motor_speed, motor_speed))
           
    prev_state="local"
    
    while not saw_obstacle:
        
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state=="local": 
                prev_state="global"
            saw_obstacle = True
            await client.sleep(0.1) #otherwise, variables would not be updated
        else:
            start = time.time()
            if (positions[n][0]+position_threshold > x_est[0][0]) and (positions[n][0]-position_threshold < x_est[0][0]):
                if (positions[n][1]+position_threshold > x_est[1][0]) and (positions[n][1]-position_threshold < x_est[1][0]):
                    n+=1
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[n],x_est[2][0]);
            print("-------angle--------",angle)
            if (abs(angle)<angle_threshold):
                #Vr = 100
                #Vl = 100
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
            else:
                if angle>0:
                    right = 1
                    #Vr = -100
                    #Vl = 100
                else:
                    right = 0
                    #Vr = 100
                    #Vl = -100

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
            stop = time.time()
            
    return 


def test_saw_obstacle(threshold, verbose=True):
    print("\t Test saw obstacle: ")
    if any([x > threshold for x in node['prox.horizontal'][0:5]]):
        if verbose: print("\t\t Obstacle detected")
        return True
    
    return False

async def local_avoidance(motor_speed=100, threshold=0, verbose=False):
    obstacleSpeedGain = [6, 4, -2, -6, -8]
    saw_obstacle = True
    speedL = motor_speed
    speedR = motor_speed
    prev_state = "global"

    if verbose:
        print("Local avoidance")

    while saw_obstacle:
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state == "global":
                prev_state = "local"
                print("\t Transitioning to local avoidance")

            for i in range(5):
                speedL += node["prox.horizontal"][i] * obstacleSpeedGain[i] // 100
                speedR += node["prox.horizontal"][i] * obstacleSpeedGain[4 - i] // 100

          
            await node.set_variables(motors(speedL, speedR))
            if verbose:
                print("\t\t Adjusting for obstacle")

        else:
            saw_obstacle = False
            if prev_state == "local":
                #prev_state = "global"
                print(prev_state)
                print("\t Transitioning to global navigation")
                
            

            # Reset motor speeds when no obstacles are detected
            speedL = motor_speed
            speedR = motor_speed
            await node.set_variables(motors(speedL, speedR))
            
            # Add a small delay to avoid continuously checking for obstacles too quickly
            await client.sleep(0.1)

async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

def camera_acquisition():
    global frame, image_ready

    ret, frame = cap.read()
    image_ready = True

def filter():
    global x_est, P_est, Ts, angle, Vr, Vl, frame, image_ready

    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]}) 

    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]

    camera_x = 0
    camera_y = 0
    camera_angle = 0

    if image_ready:
        thymio_result = locate_thymio(frame)
        camera_x = thymio_result[0][0][0]*0.4
        camera_y = thymio_result[0][0][1]*0.4
        camera_angle = (thymio_result[2])*360/(2*np.pi)
        image_ready = False

    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x, camera_y, camera_angle)
    camera_x = 0 

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    if abs(Vr-Vl) < 30:
        speed_conv_factor = speed_conv_factor_straight
    elif Vr<Vl:
        speed_conv_factor = speed_conv_factor_right
    else:
        speed_conv_factor = speed_conv_factor_left
    
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);

    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
        
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est


In [8]:
thymio_data = []
initial_orientation = 0 #degree

global image_ready
image_ready = False

# Read the first frame from the video capture
ret, init_frame = cap.read()

# Applying vision functions
thymio_result = locate_thymio(init_frame)
obstacles_result = locate_static_obstacles(init_frame, 150)
goal_result = locate_goal(init_frame)  

start = Point(thymio_result[0][0][0], thymio_result[0][0][1])
goal = Point(goal_result[0], goal_result[1])
# table_origin = Point(table_origin_result[0], table_origin_result[1])

# 
polys = []
poly = []
n_polys=0
for polygone in obstacles_result:
    n_polys += 1
    for corner in polygone:
        poly.append(Point(corner[0],corner[1]))
    polys.append(poly)
    poly = []

#polys = [[Point(170, 599), Point(170,731), Point(291,731), Point(291, 599)], [Point(431, 432), Point(398,489), Point(431, 550), Point(501, 550), Point(535, 491), Point(503, 435)], [Point(78, 129), Point(78,420), Point(531, 420), Point(531,129)]]
#start = Point(76, 81)
#goal = Point(498, 762)

positions = global_navigation(polys, start, goal, [1920,1080]) * 0.4
initial_orientation = thymio_result[2]*360/(2*np.pi)

print("Initial position :", positions/0.4)

#print(positions[0][0])
#positions = np.array([[0,0],[200,100],[10,30]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 10 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
Vr = 0
Vl = 0
position_threshold = 10
ret = 0

n = 1
await node.wait_for_variables() # wait for Thymio variables values
rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
rt_camera = RepeatedTimer(Ts, camera_acquisition)#, x_est, P_est, Ts)

try:
    print("entered try")
    for i in range(10):
        await path_following(verbose=True)
        await local_avoidance(verbose=True)
    #node.send_set_variables(motors(0, 0))
finally:
    rt.stop()
    node.send_set_variables(motors(0, 0))


/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Initial position : [[1662.57575758  477.13636364]
 [ 265.          458.        ]]
entered try
	 Path following 
	 Test saw obstacle: 
-------angle-------- -0.4649924236580887
	 Test saw obstacle: 
-------angle-------- -0.26323360359077697
	 Test saw obstacle: 
-------angle-------- -6.070948101201594
	 Test saw obstacle: 
-------angle-------- -355.46115007597905
	 Test saw obstacle: 
-------angle-------- -356.606199379184
	 Test saw obstacle: 
-------angle-------- -356.41111126908135
	 Test saw obstacle: 
-------angle-------- -349.8137691510947
	 Test saw obstacle: 
-------angle-------- -339.95436709089734
	 Test saw obstacle: 
-------angle-------- -331.8500542134859
	 Test saw obstacle: 
-------angle-------- -318.22097060268317
	 Test saw obstacle: 
-------angle-------- -312.02415833460657
	 Test saw obstacle: 
-------angle-------- -304.0431741429755
	 Test saw obstacle: 
-------angle-------- -293.5958134636036
	 Test saw obstacle: 
-------angle-------- -284.5388402108367
	 Test saw ob

CancelledError: 